In [1]:
import pandas as pd
import numpy as np
from typing import List

# Download CSV files

In [2]:
df_learn = pd.read_csv('rozetka_learn.csv')
df_test = pd.read_csv('rozetka_test.csv')

# Look on the data

In [3]:
print(f"Learn set shape: {df_learn.shape}")
print(f"Test set shape: {df_test.shape}")

Learn set shape: (3800, 5)
Test set shape: (1629, 5)


In [4]:
df_learn.head()

,goods_code,stars,review,author,permalink
0,13252316,1,Шановні! Де обіцяна знижка 5% при оплаті карто...,Андрей Дмитриев,#tab=comments;id=34200034
1,61270053,5,"Дизайн зайде не кожному, но є аналог Xiaomi Re...",Виталий Александрович,#tab=comments;id=35938497
2,45499672,5,колір тільки синій чи чорний теж є?,Арт Лаз,#tab=comments;id=30229184
3,48116526,4,"Швидкий та зручний телефон, має презентабельни...",Дима Лисунов,#tab=comments;id=36726816
4,55379058,5,Купувався мамі на подарунок. Дуже задоволена. ...,Микола,#tab=comments;id=38966133


In [5]:
df_learn.groupby('stars').count()

,goods_code,review,author,permalink
stars,,,,
1,173,173,173,173
2,122,122,122,122
3,222,222,222,222
4,746,746,746,746
5,2537,2537,2537,2537


In [6]:
df_test.groupby('stars').count()

,goods_code,review,author,permalink
stars,,,,
1,93,93,93,93
2,59,59,59,59
3,101,101,101,101
4,316,316,316,316
5,1060,1060,1060,1060


# Sentiment analizer - baseline

In [7]:
def stars_to_sentiment(stars):
    res = []
    for s in stars:
        if s<3:
            res.append('neg')
        elif s<5:
            res.append('ind') # indifferent
        else:
            res.append('pos')
    return res

def find_classes(label: str, sentiments: List[str])->List[bool]:
    return [s==label for s in sentiments]

print(stars_to_sentiment([1,2,3,4,5]))
print(find_classes('ind', stars_to_sentiment([1,2,3,4,5])))

['neg', 'neg', 'ind', 'ind', 'pos']
[False, False, True, True, False]


In [8]:
X_learn = df_learn['review'].values
X_test = df_test['review'].values
y_learn = stars_to_sentiment(df_learn['stars'].values)
y_test = stars_to_sentiment(df_test['stars'].values)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import SGDClassifier

In [17]:
# pip install git+https://github.com/kmike/pymorphy2.git pymorphy2-dicts-uk
import pymorphy2
from tone.sw import UK_STOP_WORDS
morph = pymorphy2.MorphAnalyzer(lang='uk')
norm = morph.parse('таку')[0].normal_form
print(norm)

print(list(UK_STOP_WORDS)[:10])



такий
['твоя', 'ледве', 'те', 'менше', 'можна', 'міг', 'сама', 'кілька', 'яких', 'одинадцять']


In [18]:
from nltk import word_tokenize

def tokenize_lemmatize_filter(text: str, trace: False)->List[str]:
    if trace:
        print(text)
    tokens = word_tokenize(text)
    if trace:
        print(tokens)
    #tokens = [t for t in tokens if t.lower() not in UK_STOP_WORDS]
    #tokens = [t for t in tokens if t.isalpha()]
    if trace:
        print(tokens)
    result = [morph.parse(t)[0].normal_form for t in tokens]
    if trace:
        print(result)
    return result

In [19]:
tokenize_lemmatize_filter(X_learn[0], trace=True)

Шановні! Де обіцяна знижка 5% при оплаті картою МастерКард? Раніше була а тепер зникла. Перевірте будь ласка - коли додаєш товар у кошик і обираєш "Акція! Знижка 5% при оплаті карткою Mastercard через Masterpass!" то ціна не змінюється.
['Шановні', '!', 'Де', 'обіцяна', 'знижка', '5', '%', 'при', 'оплаті', 'картою', 'МастерКард', '?', 'Раніше', 'була', 'а', 'тепер', 'зникла', '.', 'Перевірте', 'будь', 'ласка', '-', 'коли', 'додаєш', 'товар', 'у', 'кошик', 'і', 'обираєш', '``', 'Акція', '!', 'Знижка', '5', '%', 'при', 'оплаті', 'карткою', 'Mastercard', 'через', 'Masterpass', '!', "''", 'то', 'ціна', 'не', 'змінюється', '.']
['Шановні', '!', 'Де', 'обіцяна', 'знижка', '5', '%', 'при', 'оплаті', 'картою', 'МастерКард', '?', 'Раніше', 'була', 'а', 'тепер', 'зникла', '.', 'Перевірте', 'будь', 'ласка', '-', 'коли', 'додаєш', 'товар', 'у', 'кошик', 'і', 'обираєш', '``', 'Акція', '!', 'Знижка', '5', '%', 'при', 'оплаті', 'карткою', 'Mastercard', 'через', 'Masterpass', '!', "''", 'то', 'ціна', 

['шановний',
 '!',
 'де',
 'обіцяний',
 'знижка',
 '5',
 '%',
 'перти',
 'оплата',
 'карта',
 'мастеркард',
 '?',
 'раніший',
 'булий',
 'а',
 'тепер',
 'зниклий',
 '.',
 'перевірити',
 'бути',
 'ласка',
 '-',
 'коли',
 'додавати',
 'товар',
 'у',
 'кошик',
 'і',
 'обирати',
 '``',
 'акція',
 '!',
 'знижка',
 '5',
 '%',
 'перти',
 'оплата',
 'картка',
 'mastercard',
 'через',
 'masterpass',
 '!',
 "''",
 'то',
 'ціна',
 'не',
 'змінюватися',
 '.']

In [23]:
def show_most_freq_terms(features, vocab):
    # https://medium.com/@cristhianboujon/how-to-list-the-most-common-words-from-text-corpus-using-scikit-learn-dad4d0cab41d
    sum_words = features.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vocab.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    print(words_freq[:100])

def classifier_nb_tfidf(X_learn, y_learn, X_test, y_test, 
                        trace = False):
    def tokenizer(text):
        return tokenize_lemmatize_filter(text, trace=False)
    featurizer = CountVectorizer(
        tokenizer=tokenizer,
        min_df = 10, max_df = 0.1
                    ) 
    features_learn = featurizer.fit_transform(X_learn)
    if trace:
        show_most_freq_terms(features_learn, featurizer.vocabulary_)
    features_test = featurizer.transform(X_test)
    if trace:
        print(f"Dictionary size = {len(featurizer.vocabulary_)}")
        #print(sorted([(v, k) for k, v in featurizer.vocabulary_.items()], reverse=True)[:100])
    clf = BernoulliNB() # SGDClassifier()
    clf.fit(features_learn, y_learn)
    y_predicted = clf.predict(features_test)
    return y_predicted

In [24]:
from sklearn import metrics
from scipy import stats

def show_metrics_one_label(label: str, y_pred: List[bool], y_test: List[bool]):
    print(f'Metrics for {label}:')
    # print(y_pred[:20])
    # print(y_test[:20])
    print(f"  Accuracy:  {metrics.accuracy_score(y_test, y_pred)}")
    print(f"  Precision: {metrics.precision_score(y_test, y_pred)}")
    print(f"  Recall:    {metrics.recall_score(y_test, y_pred)}")
    f1 = metrics.f1_score(y_test, y_pred)
    print(f"  F1:        {f1}")
    return f1

def harmonic_mean_f1(y_pred: List[str], y_test: List[str])->float:
    f1 = metrics.f1_score(y_test, y_pred)
    hmean_f1 = stats.hmean(f1)
    
def show_metrics(y_pred: List[str], y_test: List[str]):
    f1s = []
    for label in ['neg', 'ind', 'pos']:
        f1s.append(show_metrics_one_label(label, find_classes(label, y_pred), find_classes(label, y_test)))
    print(f"Mean F1 = {np.mean(f1s)}")



In [25]:
predictions = classifier_nb_tfidf(X_learn, y_learn, X_test, y_test, trace=True)
show_metrics(list(predictions), list(y_test))


[('смартфон', 492), (':', 476), ("''", 471), ('то', 463), ('...', 449), ('ще', 421), ('без', 414), ('від', 412), ('просто', 404), ('йога', 399), ('``', 393), ('можна', 391), ('проблема', 383), ('той', 381), ('якщо', 380), ('після', 372), ('один', 371), ('том', 367), ('через', 360), ('час', 357), ('2', 357), ('розетка', 355), ('чудовий', 339), ('швидко', 335), ('супер', 327), ('звук', 325), ('тримати', 321), ('коли', 315), ('даний', 315), ('користування', 314), ('скло', 313), ('якість', 312), ('мати', 311), ('немає', 309), ('модель', 306), ('добре', 301), ('навіть', 297), ('тільки', 296), ('інший', 294), ('два', 291), ('доставка', 290), ('мій', 288), ('рекомендувати', 287), ('фото', 283), ('раз', 283), ('прийти', 276), ('рука', 270), ('робот', 264), ('швидкий', 261), ('тиждень', 260), ('гріш', 256), ('про', 255), ('перший', 255), ('тут', 249), ('також', 244), ('гарний', 241), ('чохол', 235), ('дніти', 235), ('купувати', 234), ('дно', 232), ('дякувати', 230), ('%', 228), ('більше', 228),

Я трохи погрався із параметрами, серед яких:

* Чи відфільтровувати стоп-слова
* Чи відфільтровувати токени з "небуквами"
* Параметри min_df та max_df в CountVectorizer

Найкраще значення гармонічного середнього F1 одержано при таких параметрах:

Не відфільтровуємо стоп-слова та пропускаємо всі типи токенів

min_df = 10, max_df = 0.15

Mean F1 = 0.49569680981519443